# Pré-processamento de Dados
## Praticando limpeza de dados com pandas

### Introdução
Bem-vindos ao nosso tutorial sobre pré-processamento de dados na prática com as bibliotecas Python Pandas e Scikit-learn. Hoje vamos mergulhar nos passos essenciais para limpar e preparar seus dados, garantindo análises precisas e uma construção de modelo eficaz.

Vamos começar explorando um conjunto de dados fictícios, especialmente criado com erros, para simular os desafios comuns encontrados em dados reais.

### A base de dados e o Pandas

Iniciamos importando o Pandas, com o apelido comum 'pd'. Vamos ler nosso conjunto de dados de um arquivo CSV, o qual pode ser baixado no github do livro: https://github.com/pnferreira/aprendizado-maquina,  e verificar as primeiras linhas e informações dos dados.

A base de dados apresenta diversos produtos, assim como preço, quantidade, categoria e avaliação. Perceber os valores NaN e os tipos de dados? São problemas comuns que vamos resolver hoje.


In [2]:
import pandas as pd
df = pd.read_csv("./dados_ecommerce.csv")
df.head()

,Produto,Preço,Quantidade,Categoria,Avaliação
0,Produto 1,193.52,50,Vestuário,3
1,Produto 2,475.85,6,Alimentos,1
2,Produto 3,368.68,20,NaN,s/n
3,Produto 4,303.34,72,Vestuário,3
4,Produto 5,86.45,38,Vestuário,s/n


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Produto     32 non-null     object 
 1   Preço       31 non-null     float64
 2   Quantidade  32 non-null     int64  
 3   Categoria   25 non-null     object 
 4   Avaliação   29 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.4+ KB


### Limpeza de dados – valores ausentes

A limpeza de dados é uma etapa crítica no processo de análise de dados. Nosso objetivo é remover ou corrigir pontos de dados incompletos, incorretos ou irrelevantes. Com o Pandas, há uma variedade de ferramentas à nossa disposição. Agora vamos focar no tratamento de valores ausentes, uma realidade comum em conjuntos de dados do mundo real. Em nosso conjunto de dados, cada coluna com valores ausentes exige uma abordagem específica. Começamos com a coluna de preço.

Para preços ausentes, optamos por uma solução direta: removemos as linhas inteiras onde esses valores estão faltando. Isso é feito com o comando 'dropna', especificando a coluna de interesse.



In [4]:
df.dropna(subset=['Preço'], inplace=True)

Para a coluna de categoria, a abordagem é um pouco diferente. Em vez de excluir linhas, preenchemos os espaços vazios com a etiqueta 'Sem Categoria' para manter a integridade dos dados.

In [5]:
df['Categoria'] = df['Categoria'].fillna('Sem Categoria')

Já na coluna de avaliação, encontramos o valor 's/n', que representa uma avaliação não realizada. Vamos substituir 's/n' por NaN para uniformizar os valores ausentes.
Criamos uma nova categoria para entender se há ou não avaliação, com True ou False.


In [6]:
import numpy as np
df['Avaliação'] = df['Avaliação'].replace('s/n', np.nan)
df['Avaliado'] = ~df['Avaliação'].isnull()

A intenção é preencher esses NaNs, na coluna da avaliação em si, com a média das avaliações por categoria, mas, ao tentarmos, deparamo-nos com um problema de tipo de dado.
A coluna 'Avaliação' está como string, o que impede a execução do preenchimento por média. Isso nos leva a uma etapa subsequente de limpeza: a correção dos tipos de dados.


In [7]:
media_cat = df.groupby('Categoria')['Avaliação'].mean()
df['Avaliação'].fillna(df['Categoria'].map(media_cat), inplace=True)

TypeError: agg function failed [how->mean,dtype->object]

### Limpeza de dados – tipos de dados, dados incorretos e duplicados

Com os valores ausentes devidamente tratados em cada coluna, enfrentamos agora o desafio de corrigir os tipos de dados.
Detectamos que a coluna de avaliação, que deveria conter números, foi interpretada como texto. Isso ocorre frequentemente quando os dados vêm de fontes diversas e não padronizadas.

Para resolver isso, utilizamos o método 'pd.to_numeric', que tenta converter os valores para um formato numérico, e o argumento 'errors', configurado como 'coerce', transforma os que não podem ser convertidos em NaN.



In [8]:
df['Avaliação'] = pd.to_numeric(df['Avaliação'], errors='coerce')

Após essa correção, tentamos novamente preencher os valores ausentes de 'Avaliação' com a média das avaliações por categoria. No entanto, um novo erro de formatação de dados nos impede. Observe aqui que continuamos com o NaN mesmo depois de aplicar o comando.

Ao invés de parar, esse contratempo nos leva a uma limpeza mais profunda. É hora de corrigir os dados incorretos e inconsistentes.


In [9]:
media_cat = df.groupby('Categoria')['Avaliação'].mean()
df['Avaliação'] = df['Avaliação'].fillna(df['Categoria'].map(media_cat))

df

,Produto,Preço,Quantidade,Categoria,Avaliação,Avaliado
0,Produto 1,193.52,50,Vestuário,3.000000,True
1,Produto 2,475.85,6,Alimentos,1.000000,True
2,Produto 3,368.68,20,Sem Categoria,2.666667,False
3,Produto 4,303.34,72,Vestuário,3.000000,True
4,Produto 5,86.45,38,Vestuário,3.000000,False
6,Produto 7,38.46,3,Vestuário,5.000000,True
7,Produto 8,434.43,88,Vestuário,2.000000,True
8,Produto 9,304.55,59,Alimentos,2.000000,True
9,Produto 10,356.96,13,Alimentos,1.000000,True
10,Produto 11,20.09,-5,Eletrônicos,4.000000,True


Enfrentamos um problema de categorias não padronizadas, especificamente no eletrônicos. Padronizamos com o comando df.loc. Agora, com os tipos de dados corrigidos e as categorias padronizadas, podemos executar o comando para preencher a avaliação com média e avançar para o próximo passo do nosso pré-processamento.


In [10]:
df['Categoria'].unique()

array(['Vestuário', 'Alimentos', 'Sem Categoria', 'Eletrônicos',
       'eletronicos', 'Brinquedos'], dtype=object)

In [11]:
df.loc[df['Categoria'] == 'eletronicos', 'Categoria'] = 'Eletrônicos'

In [12]:
media_cat = df.groupby('Categoria')['Avaliação'].mean()
df['Avaliação'] = df['Avaliação'].fillna(df['Categoria'].map(media_cat))

df

,Produto,Preço,Quantidade,Categoria,Avaliação,Avaliado
0,Produto 1,193.52,50,Vestuário,3.000000,True
1,Produto 2,475.85,6,Alimentos,1.000000,True
2,Produto 3,368.68,20,Sem Categoria,2.666667,False
3,Produto 4,303.34,72,Vestuário,3.000000,True
4,Produto 5,86.45,38,Vestuário,3.000000,False
6,Produto 7,38.46,3,Vestuário,5.000000,True
7,Produto 8,434.43,88,Vestuário,2.000000,True
8,Produto 9,304.55,59,Alimentos,2.000000,True
9,Produto 10,356.96,13,Alimentos,1.000000,True
10,Produto 11,20.09,-5,Eletrônicos,4.000000,True


Os nomes estão despadronizados também, alguns estão em letras maiúsculas, outras em minúsculas, e algumas até com espaços extras. Utilizamos métodos de string do Pandas para padronizar todos os nomes, deixando todas em letras minúsculas e eliminando espaços indesejados.

In [13]:
df['Produto'] = df['Produto'].str.lower().str.replace(' ', '')

Além disso, a coluna de avaliação deve estar no intervalo entre 1 e 5, e a coluna de quantidade deve ser maior do que 0. Padronizamos isso com o comando df.clip.


In [14]:
df['Avaliação'] = df['Avaliação'].clip(lower=1, upper=5)
df['Quantidade'] = df['Quantidade'].clip(lower=0)

Dessa forma, o conjunto de dados fica cada vez mais refinado, pronto para a próxima fase: a remoção de duplicatas e a correção final de quaisquer inconsistências remanescentes.

Dados duplicados podem surgir por vários motivos durante a coleta ou como resultado de erros na fusão de conjuntos de dados. É vital identificar e remover essas duplicidades para evitar distorções na análise.

Usamos o método 'drop_duplicates' do Pandas para remover linhas que são cópias exatas de outras. Especificamos qual coluna verificar e decidimos manter a primeira ocorrência.

Note que não havia linhas completamente duplicadas no conjunto de dados, mas encontramos produtos com o mesmo nome. Optamos por remover as entradas adicionais para manter a qualidade dos dados.


In [15]:
df.drop_duplicates(subset=['Produto'], keep='first', inplace=True)

### Considerações finais

Com esses ajustes, o conjunto de dados agora está limpo e pronto para análises mais complexas e para alimentar modelos de aprendizado de máquina.

Preparar seus dados é um trabalho meticuloso e, muitas vezes, demorado, mas essencial para a ciência deles. O que cobrimos hoje é apenas a ponta do iceberg, mas, com essas técnicas, você está no caminho certo para obter insights valiosos a partir de seus dados e para construir modelos de machine learning precisos.

Até a próxima!


In [16]:
df

,Produto,Preço,Quantidade,Categoria,Avaliação,Avaliado
0,produto1,193.52,50,Vestuário,3.000000,True
1,produto2,475.85,6,Alimentos,1.000000,True
2,produto3,368.68,20,Sem Categoria,2.666667,False
3,produto4,303.34,72,Vestuário,3.000000,True
4,produto5,86.45,38,Vestuário,3.000000,False
6,produto7,38.46,3,Vestuário,5.000000,True
7,produto8,434.43,88,Vestuário,2.000000,True
8,produto9,304.55,59,Alimentos,2.000000,True
9,produto10,356.96,13,Alimentos,1.000000,True
10,produto11,20.09,0,Eletrônicos,4.000000,True
